In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import argparse
import torch.optim as optim
from torchvision import datasets, transforms
from PIL import Image
import numpy as np
import copy

import tqdm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [2]:
class DuplicatedCompose(object):
    def __init__(self, transforms):
        self.transforms = transforms
        
    def __call__(self, img):
        img1 = img.copy()
        img2 = img.copy()
        for t in self.transforms:
            img1 = t(img1)
            img2 = t(img2)
        return img1, img2

In [3]:
transform = DuplicatedCompose([
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(28, scale=(0.9, 1.1), ratio=(0.9, 1.1), interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [4]:
train_mnist = datasets.MNIST("./", train=True, download=True, transform=transform)

In [5]:
train_mnist.__getitem__(0)

((tensor([[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
            -0.4242, -

In [6]:
train_mnist.__getitem__(0)[0][1].shape

torch.Size([1, 28, 28])

In [7]:
kwargs = {'num_workers': 4, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(train_mnist, batch_size=64, shuffle=True, **kwargs)

In [8]:
i = 0

for batch_idx, (data, target) in enumerate(train_loader):
    x_q = data[0]
    x_k = data[1]
    if i > 5:
        break
    i += 1